In [ ]:
import numpy as np
from numba import jit

Definir funciones para el programa

In [ ]:
@jit(nopython = True)
def cond_contorno(x, L):
    return x%L

@jit(nopython = True)
def lennard_jones(r, n):
    
